In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import time
import json
import numpy as np

In [2]:
# recipe_links = []
# for page_i in tqdm(range(1,100+1)):
#     if page_i % 5 == 0:
#         time.sleep(1)
#     r = requests.get('https://www.tudogostoso.com.br/receitas?page='+str(page_i))
#     soup = BeautifulSoup(r.text, 'html.parser')
#     page_recipes = soup.find_all('a', class_='link row m-0')
#     for link in page_recipes:
#         recipe_links = recipe_links + link.get_attribute_list('href')

In [4]:
# recipe_links_file = open('recipe_links.json','w')
# recipe_links_file.write(json.dumps(recipe_links))

f = open('../data/recipe_links.json')
recipe_links = json.load(f)

In [5]:
# pages = pd.DataFrame()
# for page_i in tqdm(range(len(recipe_links))):
#     if page_i == 100:
#         break
#     link = recipe_links[page_i]
#     page_link = 'https://www.tudogostoso.com.br'+link
#     r = requests.get(page_link)
#     pages = pages.append({
#         'link': page_link,
#         'raw_text': r.text
#     }, ignore_index=True)

In [6]:
# pages.to_csv('pages_test.csv', index=False)
pages = pd.read_csv('../data/pages_test.csv')

In [70]:
pages['ingredients'] = np.nan
pages['ingredients'] = pages['ingredients'].astype('object')

for row in tqdm(pages.itertuples()):
    soup = BeautifulSoup(row.raw_text, 'html.parser')
    page_ingredients = soup.find_all('span', class_='p-ingredient')
    
    ingredients = []
    for ingredient in page_ingredients:
        ingredient_text = ingredient.get_text()
        ingredient_text = ingredient_text.replace('\xa0','')
        if len(ingredient_text) > 0:
            ingredients.append(ingredient_text)
    pages.at[row.Index,'ingredients'] = ingredients

    text = soup.get_text()
    
    page_title = soup.find_all('title')[0].get_text().replace('\n','')
    title = [sent for sent in soup.find_all('div', class_='recipe-title')[0].get_text().split('\n') if len(sent)>1 ][0]

    datetime = soup.find_all('time', class_='dt-duration')[0]['datetime']
    time = ' '.join([sent for sent in soup.find_all('time', class_='dt-duration')[0].get_text().split('\n') if len(sent)>0 ])

    portions = soup.find_all('data', class_='p-yield num yield')[0]['value']
    favorites = soup.find_all('data', class_='num')[1].get_text().replace('\n','')
    coments = soup.find_all('data', class_='num')[2].get_text().replace('\n','')
    
    group = soup.find_all('ol', class_='breadcrumb')[0].find_all('span')[2].get_text().replace('\n','')

    images = [img['src'] for img in soup.find_all('img', class_='pic') ]
    
    pages.at[row.Index,'text'] = text
    pages.at[row.Index,'page_title'] = page_title
    pages.at[row.Index,'title'] = title
    pages.at[row.Index,'datetime'] = datetime
    pages.at[row.Index,'time'] = time
    pages.at[row.Index,'portions'] = portions
    pages.at[row.Index,'favorites'] = favorites
    pages.at[row.Index,'coments'] = coments
    pages.at[row.Index,'group'] = group
    pages.at[row.Index,'images'] = images
    
    
    
# serie = ingredients_df.ingredient.str.replace('[^A-z+\s\u00C0-\u00FF]','')
# serie = serie.str.lower()
# serie = serie.str.split(' ').apply(lambda x: [i for i in x if len(i)>0])

100it [00:10,  9.63it/s]


In [74]:
pages.to_csv('data/recipes_data_test.csv', index=False)